In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import tarfile
import pickle
import matplotlib.pyplot as plt
import scipy.io.wavfile
import scipy.signal
from moviepy.editor import *

In [3]:
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans0.txt', encoding='utf-16')
text0 = file.read()
d = json.loads(text0)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans1.txt', encoding='utf-16')
text1 = file.read()
d1 = json.loads(text1)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans2.txt', encoding='utf-16')
text2 = file.read()
d2 = json.loads(text2)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans_rest.txt', encoding='utf-16')
text3 = file.read()
d3 = json.loads(text3)
dictionnary = d | d1
dictionnary = dictionnary | d2
dictionnary = dictionnary | d3
len(dictionnary)

2280

In [4]:
import codecs

In [5]:
json_object = json.dumps(dictionnary, indent = 2)
with codecs.open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/final.json', 'w', encoding='utf-16') as writefile:
  writefile.write(json_object)

In [6]:
!pip install bert-for-tf2
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=6306678475af0f0f8d7239485b0681f1da612d16bc6b907bc70f0d1eadb273b6
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19455 sha256=8047d542a6f53722246ea0521b1468875d1117024a994fb0ec33fd8ef75451f6
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7892 sha256=6dd7a66843c40c47ec58a343fed4f01ec9159d67e767a9ea056052cae153ab0a
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e908768

In [7]:
import os
import pandas as pd
import numpy as np
import re
import random
import math

try:
    %tensorflow_version 2.x
except Exception as ex:
    print(ex)
    pass
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Transcription

In [15]:
df = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLP.csv')
df = df[df.new_label != 'not done yet']

In [16]:
df['transcription'] = df['name'].apply(lambda x : dictionnary[(x+'.mp3')])
df

,Unnamed: 0,name,label,duree,new_label,transcription
0,0,-KHQk1_Vq69E,Autos & Vehicles,21,instructional,"As you can see, the employment rate for Ameri..."
1,1,-mG4Y2Snygfk,People & Blogs,60,instructional,So So you
2,2,-OdEfx8up7wM,People & Blogs,69,cartoon,I want to hear a manly rip. Show us what you ...
3,3,-Opbfh4wNu7Q,Entertainment,61,coocking,If we do this we'd be going in shorthanded. Y...
4,4,-A5340xbbQJU,Sports,299,interview/talks/vlogs,This is the front flip shot. Got a chance peo...
...,...,...,...,...,...,...
2275,2275,-ymc30meWzfg,Education,79,health,My name is Brady Anderson. I'm a graduate tea...
2276,2276,-h7HGWRTGqyg,Education,164,accessibility,I'm gonna staple some more. There we go. Good...
2277,2277,-XVRLTfN1WYw,Howto & Style,90,instructional,"Music you Oh, yes!"
2278,2278,-jTtH7Fq1trw,Travel & Events,185,coocking,It's a great sampler of small to small to sma...


In [17]:
file = df['name'].apply(lambda x : x+'.mp3')

In [18]:
df['new_label'].value_counts()

coocking                 438
instructional            413
accessibility            286
cartoon                  167
movie/music/adverts      155
health                   127
lecture/zoom             119
discrimination           111
interview/talks/vlogs    101
animated movie            57
daily life                36
Name: new_label, dtype: int64

In [19]:
from gensim.utils import simple_preprocess

In [20]:
df['transciption'] = df['transcription'].apply(simple_preprocess,min_len=3)
df

,Unnamed: 0,name,label,duree,new_label,transcription,transciption
0,0,-KHQk1_Vq69E,Autos & Vehicles,21,instructional,"As you can see, the employment rate for Ameri...","[you, can, see, the, employment, rate, for, am..."
1,1,-mG4Y2Snygfk,People & Blogs,60,instructional,So So you,[you]
2,2,-OdEfx8up7wM,People & Blogs,69,cartoon,I want to hear a manly rip. Show us what you ...,"[want, hear, manly, rip, show, what, you, made..."
3,3,-Opbfh4wNu7Q,Entertainment,61,coocking,If we do this we'd be going in shorthanded. Y...,"[this, going, shorthanded, yeah, you, mean, be..."
4,4,-A5340xbbQJU,Sports,299,interview/talks/vlogs,This is the front flip shot. Got a chance peo...,"[this, the, front, flip, shot, got, chance, pe..."
...,...,...,...,...,...,...,...
2275,2275,-ymc30meWzfg,Education,79,health,My name is Brady Anderson. I'm a graduate tea...,"[name, brady, anderson, graduate, teaching, as..."
2276,2276,-h7HGWRTGqyg,Education,164,accessibility,I'm gonna staple some more. There we go. Good...,"[gonna, staple, some, more, there, good, that,..."
2277,2277,-XVRLTfN1WYw,Howto & Style,90,instructional,"Music you Oh, yes!","[music, you, yes]"
2278,2278,-jTtH7Fq1trw,Travel & Events,185,coocking,It's a great sampler of small to small to sma...,"[great, sampler, small, small, small, small, s..."


In [21]:
!pip install nltk

In [22]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def stemandstop(lis):
    lemmatizer = WordNetLemmatizer()
    filtered_lis = [lemmatizer.lemmatize(w) for w in lis if not w in stop_words and len(w) > 2]
    return filtered_lis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
df['transciption'] = df['transcription'].apply(stemandstop)
df['transciption'] = df['transcription'].apply(' '.join)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [25]:
from sklearn.model_selection import train_test_split

In [30]:
X = df['transcription']
y = df['new_label']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
text_clf.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [32]:
predictions = text_clf.predict(X_test)

In [29]:
predictions[:20]

array(['Film & Animation', 'Education', 'Education', 'Education',
       'Education', 'Education', 'Education', 'Education', 'Education',
       'Education', 'Education', 'Education', 'Education', 'Education',
       'Education', 'Education', 'Education', 'Education', 'Education',
       'Education'], dtype='<U21')

In [33]:
predictions[:20]

array(['coocking', 'coocking', 'coocking', 'coocking', 'instructional',
       'coocking', 'coocking', 'coocking', 'coocking', 'coocking',
       'coocking', 'coocking', 'coocking', 'instructional', 'coocking',
       'coocking', 'coocking', 'coocking', 'coocking', 'coocking'],
      dtype='<U21')

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
accuracy_score(y_test,predictions)

0.2680722891566265

In [36]:
from sklearn.svm import SVC
svm_clf = Pipeline([
    ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
    ('clf', SVC()),
])

In [37]:
svm_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', SVC())])

In [38]:
svmpred = svm_clf.predict(X_test)

In [ ]:
accuracy_score(y_test,svmpred)

0.3386454183266932

In [39]:
accuracy_score(y_test,svmpred)

0.24849397590361447

In [40]:
svm_clftfidf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC()),
])

In [41]:
svm_clftfidf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SVC())])

In [42]:
tfsvmpred = svm_clftfidf.predict(X_test)

In [ ]:
accuracy_score(y_test,tfsvmpred)

0.42363877822045154

In [43]:
accuracy_score(y_test,tfsvmpred)

0.286144578313253

# Description écrite

In [45]:
def function_reu(x):
  try:
    return dic[x]
  except:
    return ' '
def function_test(x):
  try:
    return dictionnary[(x+ '.mp3')]
  except:
    return 'do not have'
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans0.txt', encoding='utf-16')
text0 = file.read()
d = json.loads(text0)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans1.txt', encoding='utf-16')
text1 = file.read()
d1 = json.loads(text1)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans2.txt', encoding='utf-16')
text2 = file.read()
d2 = json.loads(text2)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans_rest.txt', encoding='utf-16')
text3 = file.read()
d3 = json.loads(text3)
dictionnary = d | d1
dictionnary = dictionnary | d2
dictionnary = dictionnary | d3
len(dictionnary)
descr = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/Transcription_wav.csv')
name_video = descr['name_vid']
contenue = descr['contenue']
dic = {}
vid_name_set = set()
for vid_name, cont in zip(list(name_video), list(contenue)):
  if not 'artificial intelligence' in cont.lower():
    if not vid_name in dic.keys():
      dic[vid_name] = cont
    else:
      dic[vid_name] += cont
  else:
    vid_name_set.add(vid_name)
    print(vid_name)
descr['descr'] = descr['name_vid'].apply(lambda x : function_reu(x))
descr = descr.drop_duplicates(subset='name_vid')
dt = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVT.csv')
dt['name'] = dt['video'].apply(lambda x : x[5:-4])
dt['transcription'] = dt['name'].apply(lambda x : function_test(x))
dt = dt.drop('video', axis=1)
dt = dt[dt.transcription != 'do not have']
intersection = pd.merge(dt, descr, left_on='name', right_on='name_vid')
intersection = intersection.drop('label', axis = 1)
intersection = intersection.drop('Unnamed: 0', axis = 1)
intersection = intersection.drop('name_audio', axis = 1)
intersection = intersection.drop('id_audio', axis = 1)
intersection = intersection.drop('name_vid', axis = 1)
intersection = intersection.drop('contenue', axis = 1)
intersection['total'] = intersection['text'] + intersection['descr'] + intersection['transcription']
print(len(intersection), len(dt), len(descr))

-eOrNdBpGMv8
1551 2085 1878


In [48]:
actual_dic = {}
for number in range (50):
  path = '/content/gdrive/MyDrive/Projet_Multimedia/download/NewLabel/mod' + str(number) + '.txt'
  try:
    file = open(path, encoding='utf-16')
    json0 = file.read()
    d = json.loads(json0)
    actual_dic = actual_dic | d
  except:
    print(number)
actual_dic_names = actual_dic.keys()
len(actual_dic_names)
for number in range (3):
  path = '/content/gdrive/MyDrive/Projet_Multimedia/download/NewLabel/t' + str(number) + '.txt'
  try:
    file = open(path, encoding='utf-16')
    json0 = file.read()
    d = json.loads(json0)
    actual_dic = actual_dic | d
  except:
    print(number)
actual_dic_names = actual_dic.keys()
len(actual_dic_names)
def new_label_fct_a(x):
  try:
    i = 0
    if actual_dic[x]['labels'][i] == 'accessibility' and actual_dic[x]['labels'][i+1] != 'coocking':
      i +=1
    return actual_dic[x]['labels'][i]
  except:
    return 'not done yet'

1
2


In [54]:
dt['new_label'] = dt['name'].apply(lambda x : new_label_fct_a(x))
dt = dt[dt.new_label != 'not done yet']

In [53]:
dt['new_label'].value_counts()

coocking                 438
instructional            413
accessibility            286
cartoon                  167
movie/music/adverts      155
health                   127
lecture/zoom             119
discrimination           111
interview/talks/vlogs    101
not done yet              75
animated movie            57
daily life                36
Name: new_label, dtype: int64

In [57]:
dt

,duree,label,text,label_encoded,name,transcription,new_label
0,77.60,Entertainment,"cut to Black, Carl a man in a long tan Overcoa...",0,-dXyrxjIcWdw,This has to be the place. No cameras. That's ...,coocking
2,163.42,Education,a blindfold adult wearing green sits next to a...,2,-h7HGWRTGqyg,I'm gonna staple some more. There we go. Good...,accessibility
3,125.02,Film & Animation,"a man doing a trick in the air, Robert Downey ...",3,-eOrNdBpGMv8,You were made to be ruled. In the end... It w...,accessibility
4,171.74,Film & Animation,crisp white text Upon A moonlit background Ree...,3,-dRaKMNYG3-I,You I I'm sorry. You I I Oh You You,movie/music/adverts
5,127.85,Film & Animation,send covers his mouth as he gags again he runs...,3,-faYA_upR_a0,Hmm Hmm Hmm. Hmm. Hmm. Hmm. Hmm. Hmm. Hmm. Hm...,cartoon
...,...,...,...,...,...,...,...
2094,190.03,Entertainment,the supermarket spray logo appears on the scre...,0,--UggNH1YeRY,You're watching the Game Show Network. I watc...,movie/music/adverts
2095,55.50,Education,Barbara grabs a red cup from her right and put...,2,--kNSaU8L3mI,"For another child, even something like notici...",accessibility
2096,158.59,Entertainment,"dad Smiles his lips move when he speaks, the m...",0,-0-NBRA1aSXk,Music Dad Dad Dad. Dad. Dad. Dad. Dad has voi...,coocking
2097,108.62,People & Blogs,add 3 cups of flour a teaspoon of salt and a t...,4,--bCmuE3KxWI,Music you you you you,movie/music/adverts


In [60]:
X = dt['text']
y = dt['new_label']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [62]:
svm_clftfidf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC()),
])
svm_clftfidf.fit(X_train, y_train)
tfsvmpred = svm_clftfidf.predict(X_test)
accuracy_score(y_test,tfsvmpred)

0.32981927710843373